**Project for Automated process of Web data extraction and Monitoring**

1. This script helps automatically extracted data from Qualtrics platform and save them into the specified directory. I used ChromeDrive from selenium package to aotumate this data extraction process. The script also contains several Exception Handling techniques that commonly encountered when employing this tool to do Web Scraping.
2. After the datafiles have been stored, we unzip them and read them into pd dataframe so we could conducted subseqent merging and cleaning.
3. We create a class to contain all the methods that we need to use for our primary purpose
4. For current project, we have total 4 URLs to loop over. We stored them into a list and create 4 objects, then we can call the method that we defined within the class.






In [ ]:
# import the Python libraries
import numpy as np
import pandas as pd
from collections import defaultdict
import csv
from random import randint

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bf
from selenium import webdriver
from time import sleep
import pandas as pd
import seaborn as sns
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import zipfile
import os
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
from selenium.common.exceptions import WebDriverException
from datetime import datetime

**The following session help us to reach our goal of automatically 
extract all the data in the location defined by the URLs we provided**

In [ ]:
class QualtricExtract(object):
    
      # First Let us define objects attributes
    def __init__(self,source,username,password,url):
        self.dir = source 
        self.__username = username
        self.__password = password
        self.url = url
        options = Options()
        options.add_argument("--disable-notifications")
        options.add_argument("--headless")
        options.add_argument("start-maximized")
        options.add_experimental_option("prefs", {
        "download.default_directory": self.dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
        })
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        self.driver.implicitly_wait(0.5)

    # Create a method to let system inactive for a while
    def timePractice(self):
        sleep(randint(5,10))

    # Create a method to perform connecting to the URLs that we provided
    def WebConnect(self) -> bool:

        WebConnect = False
        start_time = self.get_current_time_in_mills()
        counter = 0 

       # this technique handles 'WebDrivereException'
        while not WebConnect and not self.is_time_out(start_time, 15):
            try:
                self.driver.get(self.url)
                WebConnect = True
            except(WebDriverException) as e:

                sleep(2)
                counter += 1
                print("WebConnenction failed")

        print("WebConnenction failed " + str(counter) + " times")

        return WebConnect



    # Create a method that call on others to execute the process
    def gotoualtrics(self):
       
        WebConnect_res = self.WebConnect()
        
        # Assert Error Message if Connecting failed 
        assert WebConnect_res, "WebConnenction failed"

        self.LoginByID()

        self.DownloadData()

        self.teardown()


    # Create a method to login into the URLs we provided
    def LoginByID(self):
        self.timePractice()
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.ID, "UserName")))
        self.__key = self.driver.find_element(By.ID, "UserName")
        WebDriverWait(self.driver, 15).until(EC.visibility_of_element_located((By.ID, "UserPassword")))
        self.__pasw = self.driver.find_element(By.ID, "UserPassword")
        self.__key.send_keys(self.__username)
        self.__pasw.send_keys(self.__password)
        self.driver.find_element(By.ID,"loginButton").click()
        self.timePractice()
        
    # Create a method that navigate us to the correct page to download the data
    def DownloadData(self):

        # Find and Click 'DATA & ANALYSIS' BUTTON
        WebDriverWait(self.driver,20).until(EC.presence_of_element_located((By.XPATH,
        '//*[@id="center"]/div[2]/div/div/div/global-wrapper-react-header/global-wrapper-top-nav/div/nav[2]/div/ul/li[4]/a/div[1]')))
        self.driver.find_element(By.XPATH,
        '//*[@id="center"]/div[2]/div/div/div/global-wrapper-react-header/global-wrapper-top-nav/div/nav[2]/div/ul/li[4]/a/div[1]').click()
        
        # The page needed to load for a while
        sleep(2)

        # Find and click the 'Export&Import' button to expand the menu
        Export_Import_ele = WebDriverWait(self.driver,20).until(EC.presence_of_element_located((By.CSS_SELECTOR,'svg[data-icontype="Download"]')))
        
        Export_Import_res = self.click_until_interactable(Export_Import_ele)
        
        assert Export_Import_res, "Clicking the 'Export&Import' button has failed"

        
        #  Find and Click the 'Export' button
        Export_ele = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[value="EXPORT_DATA_ELLIPSIS"]')))  
        
        Export_res = self.click_until_interactable(Export_ele)
        
        assert Export_res, "Clicking the 'Export' button has failed"

        
        # Scroll the pop up to the next button that we need
        self.ScrollDown()


        #  Find and Click the 'Use numeric values'

        numeric_ele = WebDriverWait(self.driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="export-data-modal-body"]/div[3]/div/fieldset/label[1]/span/input')))
        
        numeric_res = self.click_until_interactable(numeric_ele)

        assert numeric_res, "Clicking the 'Use numeric values' button has failed"

        
        #  Find and Click the 'DownLoad' button
        DownLoad_ele = WebDriverWait(self.driver,20).until(EC.presence_of_element_located((By.CSS_SELECTOR,'button[class="_1jL0W _aiLYE"]')))
        
        DownLoad_res = self.click_until_interactable(DownLoad_ele)

        assert DownLoad_res, "Clicking the 'DownLoad' button has failed"        

        self.timePractice()
         
    # Create a method to scroll the page to the end, this is hardcode so it's not recommended
    def ScrollDown(self):
        for x in range(5):
            self.driver.execute_script("window.scrollBy(0,300)")
            sleep(0.1)

    # Create a method to wait for the vanishing of some elements, this is a technique to tackle ElementNotInteractableException 
    # when your elements are being blocked by other elements        
    def wait_for_element_to_vanish(self, element: webdriver.remote.webelement.WebElement) -> bool:

        is_displayed = element.is_displayed()

        start_time = self.get_current_time_in_mills()

        time_interval_in_seconds = 5

        while is_displayed and not self.is_time_out(start_time,time_interval_in_seconds):

            is_displayed = element.is_displayed()
        
        return not is_displayed


    # Create a method to get current time in millionseconds
    def get_current_time_in_mills(self):

        td = (datetime.utcnow() - datetime(1970, 1, 1))

        current_time_in_mills = (td.microseconds + (td.seconds + td.days*86400)*10**6)//10**3

        return current_time_in_mills
        

    # Create a method to set up a time limit for us to try to execute some codes
    # if the current time exceeds the end_time that we restricted to, we stop executing the codes and reporting error
    # we use this method in a while loop to avoid the possible infinite loop
    def is_time_out(self,original_time, time_interval_in_seconds) -> bool:

        wait_time = time_interval_in_seconds * 10**3

        end_time = original_time + wait_time
        
        return (self.get_current_time_in_mills() >= end_time)
        


    # Create a method to handle ElementNotInteractableException,ElementClickInterceptedException these two common exceptions
    # It is often encountered during the element locating and clicking process, the elements have been found but not clickable
    # it is possible that there is an another element overlay it and that element has received the click, 
    # But in our case, the Exception is not caused by this reason, we only have 'element is not clickable at point' message
    # It is because in the URLs that we are workiing, aftere expand the previous menu, it needs several times to load the data, only after the data has been 
    # completely loaded the button we are looking for will show up
    # The traditional Wait.until.clickable method from Expected_Condition module is not very useful in handling exceptions like this, 
    # which is due to asynchronous programming.
    # so we create a method to handle it ourself.     
    def click_until_interactable(self, element: webdriver.remote.webelement.WebElement) -> bool:

        element_is_interactable = False

        start_time = self.get_current_time_in_mills()

        counter = 1

        if element:
            while not element_is_interactable and not self.is_time_out(start_time,20):

                try:
                    element.click()
                    element_is_interactable = True

                except(ElementNotInteractableException,ElementClickInterceptedException) as e:

                    counter += 1
        
        print("We've tried " + str(counter) + " times")

        return element_is_interactable
        
    # create a method to close all the running session and release the memory
    def teardown(self):
        self.driver.quit()


**Provided the information we need and execute the codes**

In [ ]:
path = r"C:\Users\xxxxx\Desktop\xxxxxxx"
username = "xxxxxxxxx"
password = "xxxxxxxxx"
url_lists = ['https://peabody.az1.qualtrics.com/survey-builder/SV_a63UOQB30sRN0Wh/edit?LoginAction=EditSection',
             'https://peabody.az1.qualtrics.com/survey-builder/SV_3JfvpNZUqGfFQC9/edit?LoginAction=EditSection',
             'https://peabody.az1.qualtrics.com/survey-builder/SV_3dseyi0rsBznBcO/edit?LoginAction=EditSection',
             'https://peabody.az1.qualtrics.com/survey-builder/SV_9pBhZXudyC0QaGy/edit?LoginAction=EditSection']


if __name__ == "__main__":
    obj1 = QualtricExtract(path,username,password,url_lists[0])
    obj1.gotoualtrics()
    obj2 = QualtricExtract(path,username,password,url_lists[1])
    obj2.gotoualtrics()
    obj3 = QualtricExtract(path,username,password,url_lists[2])
    obj3.gotoualtrics()
    obj4 = QualtricExtract(path,username,password,url_lists[3])
    obj4.gotoualtrics()   

**Then we create another class to Unzip all the files stored in the specified dicrectory**

In [ ]:
class Unzip(object):

    dir = path # create this class variable which you could access to it by self.dir or Unzip.dir

    # First to specified some attributes that we hope to get access later
    def __init__(self):
        self.file_list = [os.path.join(self.dir,f) for f in os.listdir(self.dir)]
        self.time_sorted = sorted(self.file_list, key = os.path.getmtime)
        self.zip_names = self.time_sorted[1:len(self.time_sorted)+1]
        self.list_dfs = {}

    # Create method to Unzip all the files
    def Unzipfiles(self):
        i = 1
        for zip_name in self.zip_names:
            list_ = []
            with zipfile.ZipFile(zip_name) as myzip:
                for file in myzip.namelist():
                    with myzip.open(file) as infile:
                        list_.append(pd.read_csv(infile))
            self.list_dfs["Folder_" + str(i)] = list_
            i += 1 


**Then we create the object and call the method**

In [ ]:
if __name__ == "__main__":
    objdf = Unzip()
    objdf.Unzipfiles()


1. objdf.list_dfs is a dictionary that we used to store all the files extrated from each zip folder as VALUE.
2. we can unpack its items and loop over them to get what we need
3. #NOTE# the dfs are stored in lists, objdf.list_dfs[key] would only return a list. So we need to get access to it by objdf.list_dfs[key][0], we first extracted all of them, then we stored them into a list for next step
4. then we can assign them as different dfs

**Assign the extracted DFs**

In [ ]:
temp = [objdf.list_dfs[key][0] for key, value in objdf.list_dfs.items()]
SIDE1, SIDE2, ETHIC1, ETHIC2 = temp

**Check Data Structure of individual dataframes**

In [ ]:
# Original shapes
print(SIDE1.shape)
print(SIDE2.shape)
print(ETHIC1.shape)
print(ETHIC2.shape)

In [ ]:
# The first two rows are not meaningful we need to drop them
# DROP the first two rows and reset the index to make starting from 0
SIDE1 = SIDE1.iloc[2:,:].reset_index(drop=True)
SIDE2 = SIDE2.iloc[2:,:].reset_index(drop=True)
ETHIC1 = ETHIC1.iloc[2:,:].reset_index(drop=True)
ETHIC2 = ETHIC2.iloc[2:,:].reset_index(drop=True)

In [ ]:
#checked the changed shape
print(SIDE1.shape)
print(SIDE2.shape)
print(ETHIC1.shape)
print(ETHIC2.shape)

**Let's look at the total number of columns for these 4 data frames since we need them merge them row-wise**

I am aware of that these data frames have different number of columns, 

so when concatenate them by row, some new columns will be introduced into the final merged data frame

In [ ]:
SIDE1.shape[1] + SIDE2.shape[1] + ETHIC1.shape[1] + ETHIC2.shape[1]

If not considering the overlapping columns shared among 4 dfs, the total numbers are: 1061

**Let's find the unique column after we stored them into a list**

In [ ]:
len(set([*SIDE1.columns.tolist(),*SIDE2.columns.tolist(),*ETHIC1.columns.tolist(),*ETHIC2.columns.tolist()]))

If considering the overlapping columns shared among 4 dfs, the total numbers are: 643

**Let's merge the dfs row-wise, the pandas would automatically filled the cell with NA if there is no values entered**

In [ ]:
Combined_df = pd.concat([SIDE1, SIDE2, ETHIC1,ETHIC2],axis=0, ignore_index=True)
print(Combined_df.shape)

As being confirmed in the previous code cell, pandas would only horizontally merge unique columns and fill NA for no ENTRY, and if the columns are common among dfs, pandas would vertically concatenate them.

**Check data info and NAs in each column**

In [ ]:
Combined_df.info()

In [ ]:
pd.set_option('display.max_rows', None) 
Combined_df.isna().sum()

As we can see here, some columns have more than 2000 NA vals, they are the unique columns the pandas added column-wise. 